# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,11.52,78,30,0.45,AU,1709588918
1,1,bodo,67.2800,14.4050,1.86,78,38,6.35,NO,1709588918
2,2,keflavik,64.0049,-22.5624,5.67,81,92,9.21,IS,1709588918
3,3,port-aux-francais,-49.3500,70.2167,6.69,80,93,14.99,TF,1709588918
4,4,palo santo,-25.5633,-59.3378,25.98,79,96,0.06,AR,1709588919


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Ensure latitude and longitude are in the correct datatype (float)
city_data_df['Lat'] = city_data_df['Lat'].astype(float)
city_data_df['Lng'] = city_data_df['Lng'].astype(float)

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity', hover_cols=['City'], tiles='OSM',
                                      frame_width=400, frame_height=300, color='Humidity', cmap='blues', alpha=0.5)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Looking into the range of conditions in the dataset
print("Temperature range:", city_data_df['Max Temp'].min(), "to", city_data_df['Max Temp'].max())
print("Wind Speed range:", city_data_df['Wind Speed'].min(), "to", city_data_df['Wind Speed'].max())
print("Cloudiness range:", city_data_df['Cloudiness'].min(), "to", city_data_df['Cloudiness'].max())

Temperature range: -37.45 to 37.46
Wind Speed range: 0.06 to 15.14
Cloudiness range: 0 to 100


In [5]:
# Narrow down cities that fit the criteria and drop any results with null values

# Define ideal weather conditions
ideal_conditions = (city_data_df['Max Temp'] <= 39.62) & \
                   (city_data_df['Max Temp'] >= -34.93) & \
                   (city_data_df['Wind Speed'] <= 13.8) & \
                   (city_data_df['Cloudiness'] <= 100
                   )

# Apply conditions to the DataFrame
ideal_cities_df = city_data_df[ideal_conditions].copy()

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Reset index after the drop
ideal_cities_df.reset_index(drop=True, inplace=True)

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,11.52,78,30,0.45,AU,1709588918
1,1,bodo,67.2800,14.4050,1.86,78,38,6.35,NO,1709588918
2,2,keflavik,64.0049,-22.5624,5.67,81,92,9.21,IS,1709588918
3,4,palo santo,-25.5633,-59.3378,25.98,79,96,0.06,AR,1709588919
4,5,harper,4.3750,-7.7169,27.39,87,47,3.74,LR,1709588919
...,...,...,...,...,...,...,...,...,...,...
581,587,petrozavodsk,61.7849,34.3469,-5.83,94,35,2.24,RU,1709589090
582,588,box elder,41.5005,-112.8764,0.70,38,71,9.09,US,1709589090
583,589,edd,13.9297,41.6925,26.76,84,44,1.58,ER,1709589091
584,590,tazacorte,28.6290,-17.9293,17.65,69,25,5.94,ES,1709589091


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,78,
1,bodo,NO,67.2800,14.4050,78,
2,keflavik,IS,64.0049,-22.5624,81,
3,palo santo,AR,-25.5633,-59.3378,79,
4,harper,LR,4.3750,-7.7169,87,
...,...,...,...,...,...,...
581,petrozavodsk,RU,61.7849,34.3469,94,
582,box elder,US,41.5005,-112.8764,38,
583,edd,ER,13.9297,41.6925,84,
584,tazacorte,ES,28.6290,-17.9293,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'type': 'accommodation',
    'limit': 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
bodo - nearest hotel: Nordlysbuen Apartments
keflavik - nearest hotel: Núpan Deluxe
palo santo - nearest hotel: No hotel found
harper - nearest hotel: Cape Palmas Investment
edinburgh of the seven seas - nearest hotel: Sea View
mahibadhoo - nearest hotel: Bird Beak Beach
waitangi - nearest hotel: Travelers Rest
punta arenas - nearest hotel: No hotel found
tazovsky - nearest hotel: Тазовчанка
college - nearest hotel: Billie's Backpackers Hostel
kanniyakumari - nearest hotel: The Seashore Hotel
west island - nearest hotel: My Island Home
alice springs - nearest hotel: Aurora Alice Springs
codrington - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
isafjordur - nearest hotel: Gamla Gistihusið
la passe - nearest hotel: Le Colombier
albany - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
brookings - nearest hotel: Sonshine Inn
genh

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,78,Villa Howden
1,bodo,NO,67.2800,14.4050,78,Nordlysbuen Apartments
2,keflavik,IS,64.0049,-22.5624,81,Núpan Deluxe
3,palo santo,AR,-25.5633,-59.3378,79,No hotel found
4,harper,LR,4.3750,-7.7169,87,Cape Palmas Investment
...,...,...,...,...,...,...
581,petrozavodsk,RU,61.7849,34.3469,94,Piter Inn
582,box elder,US,41.5005,-112.8764,38,No hotel found
583,edd,ER,13.9297,41.6925,84,No hotel found
584,tazacorte,ES,28.6290,-17.9293,69,Finca Marina BuenVivir


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, size=10, color='blue', tiles='OSM',
                                  hover_cols=['City', 'Country', 'Hotel Name'],
                                  title='Hotels close to ideal cities')

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)